In [15]:
#Загрузка всех необходимых библиотек
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from datetime import date

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
#Настройка пандаса, чтобы выводить не слишком много записей
pd.set_option('display.max_rows', 6)

In [3]:
#Загрузка и просмотр данных
train_df = pd.read_csv('notes_small.csv', header=0, index_col=0)
train_df

,text,date,author_firstname,author_lastname
7729,Татьянин день. Забросил я дневник. А жаль: за ...,1922-01-25,Михаил,Булгаков
7730,Вошел в бродячий коллектив актеров: буду играт...,1922-01-26,Михаил,Булгаков
7731,Идет самый черный период моей жизни. Мы с жено...,1922-02-09,Михаил,Булгаков
...,...,...,...,...
227599,"[В ночь с 27 на 28 января 1953 Бунин, уже изме...",1953-02-27,Иван,Бунин
227600,"Вчера Алданов рассказал, что сам Алешка Толсто...",1953-02-23,Иван,Бунин
227601,"[Последняя, найденная мною запись Бунина напис...",1953-05-02,Иван,Бунин


In [4]:
#Создание новой "метафитчи" которая состоит из двух других
train_df['author'] = train_df['author_firstname'] + train_df['author_lastname']

In [5]:
#Факторизация фитчи - чтобы она была понятна компьютеру, 
#и вывод словоря сопоставлений - чтобы нам было понятно о чем речь
authors_factor, authors = train_df['author'].factorize()
authors_dict = dict(zip(authors, np.arange(len(authors))))
authors_dict

{'АлександрТвардовский': 6,
 'АндрейБелый': 7,
 'ДаниилХармс': 5,
 'ЕвгенийШварц': 3,
 'ИванБунин': 1,
 'Леонид Андреев': 4,
 'МихаилБулгаков': 0,
 'МихаилПришвин': 2}

In [6]:
#замена старых значений на факторизованные
train_df['author'] = train_df['author'].replace(authors_dict)

In [7]:
# этот код следует использовать для факторизации фитчи даты
# dates_factor, dates = train_df['date'].factorize()
# oridnal_dates = dates.map(lambda d: date(int(d.split('-')[0]), int(d.split('-')[1]), int(d.split('-')[2])).toordinal())
# dates_dict = dict(zip(dates, oridnal_dates))
# train_df['date'] = train_df['date'].replace(dates_dict)
#train_df.to_csv('notes_date_transformed.csv')

In [8]:
#Выт
    ('vect', CountVectorizer()),
    ('vect', CountVectorizer()),аскиваем из pandas необходимые нам фитчи
texts = train_df.text.astype(str).tolist()
authors = train_df.author.tolist()

In [9]:
#Разбиваем их на test и train выборки
X_train, X_test, y_train, y_test = train_test_split(texts, authors, test_size=0.33, random_state=42)

In [29]:
#Конструируем последовательность действий работы с данными
#Сначала выдергиваем фитчи, потом тренируем классификатор
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('cls', LinearRegression()),
])
#Собственно обучение классификатора
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...e_idf=True)), ('cls', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [30]:
#Проверка классификатора на тестовой выборке
pipeline.score(X_test, y_test)

0.7531589135749468

In [14]:
#Ручная проверка, понимает ли наш классификатор Булгакова
pipeline.predict(["Сегодня в газетах бюллетень о состоянии здоровья Л.Д. Троцкого. Начинается словами: «Л.Д. Троцкий 5 ноября прошлого года болел...», кончается: «Отпуск с полным освобождением от всяких обязанностей, на срок не менее 2 месяцев». Комментарии к этому историческому бюллетеню излишни."])

array([0])